In [1]:
import pandas as pd
import gzip
import os
import shutil
from geopy.geocoders import Nominatim
from geopy.point import Point
import geopandas as gpd
from shapely.geometry import Point

---

<h1><center>Bussiness Google</center></h1>

<h1><center>Lectura de información</center></h1>

El primer paso antes de realizar la normalización y limpieza de los datasets correspondientes a los metadatos de google, fue unificar los ya mencionados datasets para luego proceder a realizar la exportación de los mismos.

In [2]:
# En este bloque, realize la apertura de todos los .json de meta datos de google, para luego concatenarlos y formar un solo dataframe con toda la informacion
'''
dicc = {}

for x in range(1,12):
    dicc[x] = pd.read_json(f'extras\Google Maps\metadata-sitios\{x}.json', lines=True)
    
# Lista para almacenar los DataFrames
lista_dataframes = []

# Iterar sobre los valores del diccionario (los Datasets<.json>)
for df in dicc.values():
    lista_dataframes.append(df)

# Concatenar los DataFrames en uno solo
resultado_concatenado = pd.concat(lista_dataframes, ignore_index=True)

'''

### Cabe aclarar que este bloque de comandos solo se utiliza en primera instancia, ya que luego se trabajara sobre la exportación unificada

In [3]:
#resultado_concatenado.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [3]:
# Lectura de dataset unificado  
meta_datos_google = pd.read_json(r'..\extras\google Maps\metadata-sitios\bussiness_google.json.gz')

meta_datos_google.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [10]:
print(resultado_concatenado.columns.values)

['name' 'address' 'gmap_id' 'description' 'latitude' 'longitude'
 'category' 'avg_rating' 'num_of_reviews' 'price' 'hours' 'MISC' 'state'
 'relative_results' 'url']


## Columnas eliminadas

-state (porque no es relevante, dice si esta abierto el local o no)

-url (se puede ingresar utilizando la siguiente url mas el gmap-id) [https://www.google.com/maps/place//data=!4m2!3m1!1s {aca va el gmap_id}]

-description (esta información podemos obtenerla en caso de ser necesario usando la api, solo tiene 250.000 datos lo que vuelve a la columna irrelevante en comparación con la cantidad total, por eso mismo se recomienda realizar la extracción de dicha columna de ser necesario desde la api)

-MISC (esta columna tiene una descripcion de como estan las condiciones del local)

-price (es irrelevante el precio)

-num_of_reviews (esta columna tendrémos que volver a crearla luego de realizar el ETL de los estados)

-address (esta información podemos extraerla del url en el caso de ser necesario)

-hours (en caso de ser necesario se extraeran los datos específicos de la url)

-relative_result (podemos obtener el mismo resultado y hasta mas eficiente realizando una ponderación de los valores de latitude y longitude)

Muestra de acceso a google maps por medio de url mas gmap_id:

https://www.google.com/maps/place//data=!4m2!3m1!1s0x88e77e406e2da09d:0x479c876de5fd4ec4

In [ ]:
resultado_concatenado.drop(columns=['state','url','description','MISC','price','num_of_reviews','address','hours','relative_results'],inplace=True)

In [11]:
resultado_concatenado

,name,gmap_id,latitude,longitude,category,avg_rating
0,Porter Pharmacy,0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,[Pharmacy],4.9
1,City Textile,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.215290,[Textile exporter],4.5
2,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,[Korean restaurant],4.4
3,Nova Fabrics,0x80c2c89923b27a41:0x32041559418d447,34.023669,-118.232930,[Fabric store],3.3
4,Nobel Textile Co,0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,[Fabric store],4.3
...,...,...,...,...,...,...
3025006,Steak 48,0x89c6c74f43a49b55:0x6be6995921c58b12,39.947254,-75.164953,"[Steak house, Fine dining restaurant, Seafood ...",4.6
3025007,"Jack Mcnerney Chevrolet, Inc.",0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,42.788636,-76.122120,[Chevrolet dealer],4.4
3025008,Central Ny Spay Neuter Assista,0x89da6b9ce832726f:0xbd0d3630f1e3b42e,42.603831,-76.165286,[Veterinarian],4.3
3025009,Ok Feed Store,0x88d9e872a9e39f3d:0x5de29f50507e5d73,25.491641,-80.456076,[Animal feed store],4.9


In [70]:
bussiness_google = pd.read_json(r'..\datasets\processed\google\bussiness_google.json.gz.gz')
bussiness_google

,gmap_id,latitude,longitude,category,stars,name,state
0,0x88d9beb4fe0532c1:0xef0555c169299d6,25.795204,-80.366038,['Corporate office'],5.0,NTK OUTDOOR,Florida
1,0x88c2e49b79f06c31:0x4ed8c8ce27e926e0,27.832187,-82.704805,['Cruise agency'],5.0,Cruises Inc. - Connie Stewart,Florida
2,0x88e711bd2244fe3b:0x8406dd780f0574d1,28.877581,-81.281276,['Barber shop'],4.0,First Impressions Barbershop Inc.,Florida
3,0x8893863ea87bd5dd:0x9383ebf973e74abb,30.159982,-85.752277,['Family practice physician' 'General practiti...,4.2,"Brian Shaheen, MD",Florida
4,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,['Restaurant'],5.0,Cape Seafood Shack,Florida
...,...,...,...,...,...,...,...
693482,0x89c252b98d7621fd:0x4ac46f3bd7756984,40.683651,-74.203041,['Railroad company'],4.3,AMTRAK DURANT YARD,New Jersey
693483,0x89c1766b2682309d:0x4677876d0012f9a6,39.972538,-74.241226,['Dental clinic' 'Dentist' 'General practition...,4.9,T. R. Smiles Dental,New Jersey
693484,0x89c17b4f55ec117b:0x87fd46b1a8175b52,40.121928,-74.393946,['Barber shop'],4.7,Paisano's Barber Shop,New Jersey
693485,0x89c17b3b65b4680d:0x40a43d960a97a0,40.109785,-74.387818,['Pizza restaurant' 'Italian restaurant' 'Pizz...,4.5,Jackson Pizza,New Jersey


In [ ]:
illinois = gpd.read_file(r'..\datasets\raw\IL.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_illinois_google = gpd.sjoin(meta_datos_google, illinois, op='within')

In [ ]:
california = gpd.read_file(r'..\datasets\raw\CA.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_california_google = gpd.sjoin(meta_datos_google, california, op='within')

In [ ]:
florida = gpd.read_file(r'..\datasets\raw\FL.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_florida_google = gpd.sjoin(meta_datos_google, florida, op='within')

In [ ]:
Nuevajersey = gpd.read_file(r'..\datasets\raw\NJ.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_Nuevajersey_google = gpd.sjoin(meta_datos_google, Nuevajersey, op='within')

In [60]:
print(len(empresas_en_florida_google) + len(empresas_en_california_google) + len(empresas_en_illinois_google) + len(empresas_en_Nuevajersey_google))

693487


In [67]:
# Lista de DataFrames a concatenar
dataframes = [empresas_en_florida_google, empresas_en_california_google, empresas_en_illinois_google, empresas_en_Nuevajersey_google]

# Concatenar los DataFrames verticalmente
empresas_concatenadas_google = pd.concat(dataframes, ignore_index=True)

In [68]:
# Lista de nombres de columnas a eliminar
columnas_a_eliminar = ['geometry', 'index_right', 'id', 'fips','Estado_Perteneciente']

# Eliminar las columnas de empresas_concatenadas
empresas_concatenadas_google = empresas_concatenadas_google.drop(columns=columnas_a_eliminar)

empresas_concatenadas_google.rename(columns={'name_right': 'state'}, inplace=True)
empresas_concatenadas_google.rename(columns={'name_left': 'name'}, inplace=True)
empresas_concatenadas_google.rename(columns={'avg_rating': 'stars'}, inplace=True)

empresas_concatenadas_google.head()

,gmap_id,latitude,longitude,category,stars,name,state
0,0x88d9beb4fe0532c1:0xef0555c169299d6,25.795204,-80.366038,['Corporate office'],5.0,NTK OUTDOOR,Florida
1,0x88c2e49b79f06c31:0x4ed8c8ce27e926e0,27.832187,-82.704805,['Cruise agency'],5.0,Cruises Inc. - Connie Stewart,Florida
2,0x88e711bd2244fe3b:0x8406dd780f0574d1,28.877581,-81.281276,['Barber shop'],4.0,First Impressions Barbershop Inc.,Florida
3,0x8893863ea87bd5dd:0x9383ebf973e74abb,30.159982,-85.752277,['Family practice physician' 'General practiti...,4.2,"Brian Shaheen, MD",Florida
4,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,['Restaurant'],5.0,Cape Seafood Shack,Florida


In [ ]:
# Creo listas para utilizar en las siguientes iteraciones
categorias = []
filtro_cat = []
restaurant_cat = []

# Itero por la columna categories del dataframe, para extraer todos sus valores y convertirlos en listas, los que luego agrego a la lista categorias
for x in empresas_concatenadas_google['category']:
    alm = x.split("'")
    categorias.append(alm)
    
# Realizo una limpieza de cada lista dentro de la lista categorias
for x in categorias:
    for i in x:
        if len(i) > 3:
            filtro_cat.append(i)

# Itero sobre cada fila de la columna categories, con la finalidad de filtrar todos los comercios que corresponden a restaurantes
for x in filtro_cat:
    if 'estau' in x:
        restaurant_cat.append(x)

# Con esta conversion elimino los valores repetidos de toda mi lista
restaurant_cat = set(restaurant_cat)

# Creo un filtro con el cual selecciono las columnas coincidentes
filtro = empresas_concatenadas_google['category'].str.contains('|'.join(restaurant_cat))

# Exporto el dataframe con su nueva mascara para crear uno nuevo con los datos esperados
empresas_concatenadas_restaurantes_estados_google = empresas_concatenadas_google[filtro]

In [88]:
empresas_concatenadas_restaurantes_estados_google

,gmap_id,latitude,longitude,category,stars,name,state
4,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,['Restaurant'],5.0,Cape Seafood Shack,Florida
5,0x88c2e4e34f1ed783:0x76c5da381c499d79,27.867489,-82.702971,['Buffet restaurant'],5.0,Fresh Point Country Buffet,Florida
46,0x8890b9241e704667:0x3a1e565c17c00993,30.391411,-87.267220,['Restaurant'],4.2,Hot Box,Florida
81,0x88e635378f43352f:0xa1b53c63436fa428,29.183272,-81.889965,['Gas station' 'ATM' 'Convenience store' 'Rest...,1.8,Shell,Florida
156,0x88d9ab98b5baad79:0x1a2e3f0642a55246,26.010250,-80.148659,['Mexican restaurant'],3.0,Baby Food Grill & Bar,Florida
...,...,...,...,...,...,...,...
693466,0x89c3b2b8e9c8c357:0xb1b07826ef85a598,40.653743,-74.234148,['Bar' 'Italian restaurant' 'Pub' 'Restaurant'],4.3,Park View Tavern,New Jersey
693471,0x89c24d60acc3ffff:0xc8048546518281f6,40.661250,-74.193971,['Restaurant' 'Family restaurant'],4.6,The Brown Derby Cafe & Rental Hall,New Jersey
693475,0x89c24d7a198a3541:0x3dcb8f429c053248,40.654865,-74.177221,['American restaurant'],4.5,George's Lunch,New Jersey
693478,0x89c3b290dcc12d6d:0x34ad59938efed577,40.661939,-74.248363,['Latin American restaurant' 'Mexican restaura...,3.9,Las Brasas Sports Bar and Restaurant,New Jersey


In [77]:

# Suponiendo que 'json' es tu DataFrame
empresas_concatenadas_restaurantes_estados_google.to_json(r'..\datasets\processed\google\bussiness_google.json', index=False)

# Ruta del archivo CSV que quieres comprimir con gzip
input_file = r'..\datasets\processed\google\bussiness_google.json'

# Ruta de salida para el archivo comprimido con gzip
output_file = r'..\datasets\processed\google\bussiness_google.json.gz'

# Comprimir el archivo CSV usando gzip
with open(input_file, 'rb') as f_in:
    with gzip.open(output_file, 'wb') as f_out:
        f_out.writelines(f_in)

# Eliminar el archivo json original si se desea
os.remove(input_file)


<h1><center>Bussiness Yelp</center></h1>

<h1><center>Lectura de informacion</center></h1>

In [2]:
bussiness_yelp = pd.read_pickle(r'..\extras\yelp\business.pkl')

In [3]:
bussiness_yelp.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
bussiness_yelp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

In [5]:
# Eliminamos los duplicados de las columnas
bussiness_yelp = bussiness_yelp.iloc[::,:14].copy()

In [6]:
bussiness_yelp.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


## Columnas eliminadas:

- addres

- city

- state

- postal_code

- review_count

- is_open

- attributes

- hours

In [7]:
bussiness_yelp_filtrado = bussiness_yelp[['business_id', 'name', 'latitude', 'longitude', 'stars', 'categories']]
bussiness_yelp_filtrado

,business_id,name,latitude,longitude,stars,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",34.426679,-119.711197,5.0,"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,38.551126,-90.335695,3.0,"Shipping Centers, Local Services, Notaries, Ma..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,32.223236,-110.880452,3.5,"Department Stores, Shopping, Fashion, Home & G..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,39.955505,-75.155564,4.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,40.338183,-75.471659,4.5,"Brewpubs, Breweries, Food"
...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,53.468419,-113.492054,3.0,"Nail Salons, Beauty & Spas"
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,36.115118,-86.766925,4.0,"Pets, Nurseries & Gardening, Pet Stores, Hobby..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,39.908707,-86.065088,3.5,"Shopping, Jewelry, Piercing, Toy Stores, Beaut..."
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,38.782351,-89.950558,4.0,"Fitness/Exercise Equipment, Eyewear & Optician..."


In [ ]:
florida = gpd.read_file(r'..\datasets\raw\FL.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_yelp_filtrado['longitude'], bussiness_yelp_filtrado['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_yelp_filtrado, geometry=geometry)
# Realizar la unión espacial
empresas_en_florida = gpd.sjoin(meta_datos_google, florida, op='within')

In [ ]:
california = gpd.read_file(r'..\datasets\raw\CA.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_yelp_filtrado['longitude'], bussiness_yelp_filtrado['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_yelp_filtrado, geometry=geometry)
# Realizar la unión espacial
empresas_en_california = gpd.sjoin(meta_datos_google, california, op='within')

In [ ]:
illinois = gpd.read_file(r'..\datasets\raw\IL.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_yelp_filtrado['longitude'], bussiness_yelp_filtrado['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_yelp_filtrado, geometry=geometry)
# Realizar la unión espacial
empresas_en_illinois = gpd.sjoin(meta_datos_google, illinois, op='within')

In [ ]:
Nuevajersey = gpd.read_file(r'..\datasets\raw\NJ.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_yelp_filtrado['longitude'], bussiness_yelp_filtrado['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_yelp_filtrado, geometry=geometry)
# Realizar la unión espacial
empresas_en_Nuevajersey = gpd.sjoin(meta_datos_google, Nuevajersey, op='within')

In [78]:
print(len(empresas_en_florida) + len(empresas_en_california) + len(empresas_en_illinois) + len(empresas_en_Nuevajersey))

109313


In [21]:
# Lista de DataFrames a concatenar
dataframes = [empresas_en_florida, empresas_en_california, empresas_en_illinois, empresas_en_Nuevajersey]

# Concatenar los DataFrames verticalmente
empresas_concatenadas = pd.concat(dataframes, ignore_index=True)

In [29]:
# Visualizar las columnas del dataframe
empresas_concatenadas.columns

Index(['business_id', 'name_left', 'latitude', 'longitude', 'stars',
       'categories', 'geometry', 'index_right', 'id', 'fips', 'name_right'],
      dtype='object')

In [37]:
# Lista de nombres de columnas a eliminar
columnas_a_eliminar = ['geometry', 'index_right', 'id', 'fips']

# Eliminar las columnas de empresas_concatenadas
empresas_concatenadas = empresas_concatenadas.drop(columns=columnas_a_eliminar)

# Renombro las columnas que considero pertinentes
empresas_concatenadas.rename(columns={'name_right': 'Estado_Perteneciente'}, inplace=True)
empresas_concatenadas.rename(columns={'name_left': 'name'}, inplace=True)

empresas_concatenadas.head()

,business_id,name,latitude,longitude,stars,categories,Estado_Perteneciente
0,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,27.76659,-82.732983,3.5,"Synagogues, Religious Organizations",Florida
1,UJsufbvfyfONHeWdvAHKjA,Marshalls,28.190459,-82.45738,3.5,"Department Stores, Shopping, Fashion",Florida
2,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,27.955269,-82.45632,4.0,"Vietnamese, Food, Restaurants, Food Trucks",Florida
3,jaxMSoInw8Poo3XeMJt8lQ,Adams Dental,27.966235,-82.787412,5.0,"General Dentistry, Dentists, Health & Medical,...",Florida
4,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,27.916116,-82.760461,4.5,"Food, Delis, Italian, Bakeries, Restaurants",Florida


In [89]:
# Eliminamos los valores nulos para evitar errores a largo plazo
empresas_concatenadas.dropna(inplace=True)

# Creo listas para utilizar en las siguientes iteraciones
categorias = []
filtro_cat = []
restaurant_cat = []

# Itero por la columna categories del dataframe, para extraer todos sus valores y convertirlos en listas, los que luego agrego a la lista categorias
for x in empresas_concatenadas['categories']:
    alm = x.split(",")
    categorias.extend(alm)

# Realizo una limpieza de cada lista dentro de la lista categorias
for i in categorias:
    if len(i.strip()) > 3:
        filtro_cat.append(i.strip())

# Itero sobre cada fila de la columna categories, con la finalidad de filtrar todos los comercios que corresponden a restaurantes
for x in filtro_cat:
    if 'estau' in x.lower():  # Convertimos a minúsculas para hacer la búsqueda insensible a mayúsculas/minúsculas
        restaurant_cat.append(x)

# Con esta conversion elimino los valores repetidos de toda mi lista
restaurant_cat = set(restaurant_cat)

# Creo un filtro con el cual selecciono las columnas coincidentes
filtro = empresas_concatenadas['categories'].str.contains('|'.join(restaurant_cat))

# Exporto el dataframe con su nueva mascara para crear uno nuevo con los datos esperados
empresas_concatenadas_restaurantes_estados_yelp = empresas_concatenadas[filtro]

# Prueba de visualizacion
empresas_concatenadas_restaurantes_estados_yelp

,business_id,name,latitude,longitude,stars,categories,Estado_Perteneciente
2,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,27.955269,-82.456320,4.0,"Vietnamese, Food, Restaurants, Food Trucks",Florida
4,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,27.916116,-82.760461,4.5,"Food, Delis, Italian, Bakeries, Restaurants",Florida
7,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,28.046203,-82.505053,4.0,"Restaurants, American (New), Italian",Florida
8,JgpnXv_0XhV3SfbfB50nxw,Joe's Pizza,27.960514,-82.506127,4.0,"Restaurants, Pizza",Florida
12,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,28.196252,-82.380615,4.5,"Burgers, Sports Bars, Bars, Lounges, Restauran...",Florida
...,...,...,...,...,...,...,...
41313,qjtELTt9fdIwoi_xGNN21g,Chipotle Mexican Grill,39.943481,-74.964743,3.0,"Restaurants, Mexican, Fast Food",New Jersey
41316,BzPTw9daJW8ToqTc7QiHFw,Taco Bell,40.210771,-74.756588,2.5,"Restaurants, Tex-Mex, Fast Food, Mexican, Taco...",New Jersey
41319,oNos27QR8hkBb4z-IK6uTA,Cafe Tango,39.847724,-74.977354,2.5,"Indian, Restaurants, Tapas/Small Plates",New Jersey
41320,UBQAksw81m0sMrAd8g-ECg,El Zarape Restaurant,39.718717,-74.971143,4.0,"Restaurants, Mexican",New Jersey


In [90]:
#Normalizamos los nombres de las columnas y actualizo index
empresas_concatenadas_restaurantes_estados_yelp.columns = ['business_id', 'name', 'latitude', 'longitude', 'stars', 'categories', 'state']
empresas_concatenadas_restaurantes_estados_yelp.reset_index(drop=True, inplace=True)

empresas_concatenadas_restaurantes_estados_yelp.to_parquet(r'..\datasets\processed\yelp\bussiness_yelp.parquet.gz', compression='gzip')

---

<h1><center>Fin de ETL correspondiente a Bussines Google/Yelp</center</h1>

#### A continuación se con la información normalizada se procede a realizar el análisis exploratiorio de los mismos, con la finalidad de localizar diferentes metricas a establecer como KPIS, este proceso de limpieza y normalización, nos da como parametros los datasets filtrados por Estados que fueron previamente seleccionados y solo las categorías de restaurante.

---